## Test lấy 10000 bộ phim lưu trực tiếp vào cloud sql server

In [ ]:
import time
import random
import pyodbc
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, TimeoutException, StaleElementReferenceException


# Cấu hình Selenium
options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option('excludeSwitches', ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)

# Danh sách các User-Agent phổ biến
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
]

# Chọn ngẫu nhiên 1 User-Agent từ danh sách
options.add_argument(f"user-agent={random.choice(user_agents)}")

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

# Kết nối tới Cloud SQL Server
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=34.87.28.157;'
    r'DATABASE=imdb;'
    r'UID=sqlserver;'
    r'PWD=1234567890;'
)
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Hàm reconnect để kết nối lại khi bị mất kết nối
def reconnect():
    while True:
        try:
            conn = pyodbc.connect(conn_str)
            cursor = conn.cursor()
            print("Kết nối lại thành công")
            return conn, cursor
        except pyodbc.OperationalError as e:
            print("Lỗi kết nối, đang thử lại...")
            time.sleep(5)  

# Kiểm tra và tạo bảng nếu chưa tồn tại trong SQL Server
create_table_query = '''
IF NOT EXISTS (
    SELECT * FROM INFORMATION_SCHEMA.TABLES 
    WHERE TABLE_NAME = 'Movies'
)
BEGIN
    CREATE TABLE Movies (
        Title NVARCHAR(255),
        Year INT,
        Duration NVARCHAR(10),
        MPAA NVARCHAR(10),
        Genres NVARCHAR(255),
        IMDb_Rating DECIMAL(3,1),
        Director NVARCHAR(100),
        Stars NVARCHAR(255),
        Plot_Summary NVARCHAR(MAX),
        CONSTRAINT unique_movie UNIQUE (Title)
    );
END
'''

cursor.execute(create_table_query)
conn.commit()

# URL cần crawl
url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2013-01-01,2024-10-05&user_rating=6.5,10&languages=en"
driver.get(url)

# Số lần cần click vào "Load More" để tải đủ 10,000 bộ phim
load_more_clicks_needed = 199
current_clicks = 0

while current_clicks < load_more_clicks_needed:
    try:
        load_more_button = wait.until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.ipc-see-more__button'))
        )
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more_button)
        time.sleep(1)
        load_more_button.click()
        current_clicks += 1
        time.sleep(random.uniform(2, 4))  

    except TimeoutException:
        print(f"Không tìm thấy nút '50 more' sau lần click thứ {current_clicks + 1}. Dừng việc click.")
        break

# Sau khi click xong, lấy danh sách các bộ phim
movies_list = driver.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item')

# Bắt đầu thu thập dữ liệu từ phim thứ 1
movies_data = []
total_movies_to_crawl = 10000
i = 0

while i < total_movies_to_crawl and i < len(movies_list):
    try:
        movie = movies_list[i]

        # Cuộn tới bộ phim để lấy thông tin
        # Sử dụng vòng lặp để thử lại nếu gặp lỗi StaleElementReferenceException
        stale_attempts = 0
        while stale_attempts < 3:  # Thử lại tối đa 3 lần
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", movie)
                time.sleep(random.uniform(1, 2))  # Nghỉ ngẫu nhiên để tránh bị phát hiện là bot
                break  # Nếu không có lỗi, thoát khỏi vòng lặp
            except StaleElementReferenceException:
                print(f"Lỗi StaleElementReferenceException, thử lại lần {stale_attempts + 1}")
                stale_attempts += 1
                # Làm mới danh sách phần tử và thử lại
                movies_list = driver.find_elements(By.CSS_SELECTOR, 'li.ipc-metadata-list-summary-item')
                movie = movies_list[i]

        # Tìm nút "See more information" và click
        info_button = movie.find_element(By.CSS_SELECTOR, 'button[title^="See more information"]')
        info_button.click()
        time.sleep(random.uniform(1, 2))

        # Kiểm tra xem popup đã mở hay chưa
        if EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="dialog"]')):
            popup_container = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[role="dialog"]')))
        else:
            print(f"Không có popup container cho phim thứ {i + 1}, bỏ qua.")
            continue

        # Lấy các thông tin về phim
        title = popup_container.find_element(By.CSS_SELECTOR, 'h3.ipc-title__text.prompt-title-text').text
        print(f"Đang crawl phim thứ {i + 1}: {title}")


        # Lấy các thông tin khác
        try:
            info_list = popup_container.find_elements(By.CSS_SELECTOR, 'ul[data-testid="btp_ml"] li')
            year = info_list[0].text if len(info_list) > 0 else ""
            duration = info_list[1].text if len(info_list) > 1 else ""
            mpaa = info_list[2].text if len(info_list) > 2 else ""
        except NoSuchElementException:
            year, duration, mpaa = "", "", ""  

        try:
            genres = ', '.join([genre.text for genre in popup_container.find_elements(By.CSS_SELECTOR, 'ul[data-testid="btp_gl"] li.ipc-inline-list__item')])
        except NoSuchElementException:
            genres = ""

        try:
            imdb_rating = popup_container.find_element(By.CSS_SELECTOR, 'span.ipc-rating-star--rating').text
            imdb_rating = float(imdb_rating) if imdb_rating else None  
        except (NoSuchElementException, ValueError):
            imdb_rating = None

        try:
            plot_summary = popup_container.find_element(By.CSS_SELECTOR, 'div.sc-65f72df1-2.geapts').text 
        except NoSuchElementException:
            plot_summary = ""
            
        try:
            director = popup_container.find_element(By.CSS_SELECTOR, 'div.sc-1582ce06-3.iWfkOS > div:nth-child(1) > ul > li > a').text
        except NoSuchElementException:
            director = ""

        try:
            stars_elements = popup_container.find_elements(By.CSS_SELECTOR, 'div.sc-1582ce06-3.iWfkOS > div:nth-child(2) > ul li')
            stars = ', '.join([star.text for star in stars_elements if star.text != director]) if stars_elements else ""
        except NoSuchElementException:
            stars = ""
            
        # Đóng popup sau khi lấy thông tin
        try:
            close_button = popup_container.find_element(By.CSS_SELECTOR, 'button[title="Close Prompt"]')
            close_button.click()
            WebDriverWait(driver, 3).until(EC.invisibility_of_element(popup_container))
        except Exception as e:
            print(f"Lỗi khi đóng popup: {e}")
            continue
        
        try:
            votes_element = movie.find_element(By.CSS_SELECTOR, 'span.ipc-rating-star--voteCount')
            votes = votes_element.text.strip() if votes_element else ""
        except NoSuchElementException:
            votes = ""        
            
        # Lưu dữ liệu vào danh sách
        movie_data = {
            'Title': title,
            'Year': year,
            'Duration': duration,
            'MPAA': mpaa,
            'Genres': genres,
            'IMDb_Rating': imdb_rating,
            'Director': director,
            'Stars': stars,
            'Plot_Summary': plot_summary,
            'Votes': votes
        }


        movies_data.append(movie_data)

        # Lưu vào cơ sở dữ liệu mỗi khi crawl được 10 phim
        if len(movies_data) >= 10:
            try:
                for movie in movies_data:
                    insert_query = '''
                    INSERT INTO Movies (Title, Year, Duration, MPAA, Genres, IMDb_Rating, Director, Stars, Plot_Summary, Votes)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                    '''
                    cursor.execute(insert_query, movie['Title'], movie['Year'], movie['Duration'], movie['MPAA'], movie['Genres'], movie['IMDb_Rating'], movie['Director'], movie['Stars'], movie['Plot_Summary'], movie['Votes'])                
                conn.commit()
                print(f"Đã lưu thành công {len(movies_data)} bộ phim vào cơ sở dữ liệu!")
                movies_data = []  # Reset danh sách dữ liệu sau khi lưu
            except pyodbc.IntegrityError:
                # Bắt lỗi nếu gặp duplicate key
                print(f"Phim {movie['Title']} đã tồn tại trong cơ sở dữ liệu. Bỏ qua.")
            except pyodbc.OperationalError as e:
                print(f"Lỗi kết nối: {e}. Đang thử kết nối lại...")
                conn, cursor = reconnect()

        i += 1
        time.sleep(random.uniform(0.5, 2))

    except (ElementClickInterceptedException, TimeoutException) as e:
        print(f"Lỗi khi click vào phần tử: {e}")
        continue

driver.quit()
print("Crawl dữ liệu hoàn tất.")
